In [37]:
"""
generate train data, validation data, and test data
"""
import os
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.ndimage import imread
from scipy.misc import imsave
from scipy.misc import imresize
import numpy as np
import os
from numpy import genfromtxt
import tensorflow as tf
from PIL import Image
import pickle
import casia_b_LSTM_identification_helper
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
import collections
import time

# Create Train and Target Data

In [2]:
# get codes
gallery_features_batch1 = np.load(open(r'gait_data/casia_b_fc8_124/casia_b_72c_124_seq1', mode='rb'))
probe_features_batch1 = np.load(open(r'gait_data/casia_b_fc8_124/casia_b_72c_124_seq2', mode='rb'))
# feature shape
print("gallery length: {}".format(gallery_features_batch1.shape))
print("gallery shape: {}".format(gallery_features_batch1[0].shape))
print("")
print("probe length: {}".format(probe_features_batch1.shape))
print("probe shape: {}".format(probe_features_batch1[0].shape))

gallery length: (124,)
gallery shape: (53, 1000)

probe length: (124,)
probe shape: (51, 1000)


### Add Zero

In [7]:
gallery_features_batch1_add0 = np.zeros([len(gallery_features_batch1), 82, 1000], dtype=float)
probe_features_batch1_add0 = np.zeros([len(probe_features_batch1), 82, 1000], dtype=float)

g_seq_length = []
p_seq_length = []
for ii, (gallery, probe) in enumerate(zip(gallery_features_batch1, probe_features_batch1)):
    gallery_length = gallery.shape[0]
    probe_length = probe.shape[0]
    g_seq_length.append(gallery_length)
    p_seq_length.append(probe_length)
    
    gallery_features_batch1_add0[ii, :gallery_length, :] = gallery
    probe_features_batch1_add0[ii, :probe_length, :] = probe

### Create Target 1 Data

In [9]:
batch_1_target_1 = np.concatenate((gallery_features_batch1_add0, probe_features_batch1_add0), axis=1)
target_1 = np.ones([batch_1_target_1.shape[0]])

In [11]:
print("batch_1_target_1:", batch_1_target_1.shape)
print("")
print("target_1:", target_1.shape)

batch_1_target_1: (124, 164, 1000)

target_1: (124,)


### Create Target 0 Data
We create a same length batch with shuffled data
直到检测shuffle的idx没有重叠，函数会返回ok，否则打印error

In [20]:
idx = np.arange(len(gallery_features_batch1_add0))
np.random.shuffle(idx)
idx

array([ 34, 116,  61,  94,  69,   3,  59, 113,  19,  31, 110,  22,  93,
        96,  27,  14,  55,  54,  75, 123,  74, 100,  15,  57,  47,  44,
        45,  48,  60,   2,  39,   0,  46,  17,  58,  30,  82, 121,  79,
        88,   9,  87,  84, 103,  42,  35, 111,  71,  18,  41,  38, 109,
        95,  65,   1,  64, 115,  62,  23,  89,  53,  25,  29, 105,  21,
        50, 117,  13,  83,  12, 122,  72,  92, 120,   7,  20,  85, 101,
        26,  66,  49,  98,   6,  97,  32,  10,  81,  91,  33,  40,  77,
        43,  68,  52, 114,  73, 112,  51,  63,  78,  67,  99,  70,  16,
       102,  24,  36, 104,  76,  86, 107,  90, 106,  37,   4,  56,  11,
         5, 119,  28,   8,  80, 118, 108])

In [21]:
def exam_shuffle_idx(idx):
    x =None
    for ii, index in enumerate(idx):
        if ii == index:
            x ='error'
            print(x)
            break
    if x != 'error':
        print("ok")
    return None
exam_shuffle_idx(idx)

ok


In [22]:
# create shuffled probe_features_batch1
shuffled_probe_features_batch1 = np.zeros(probe_features_batch1_add0.shape)
print(shuffled_probe_features_batch1.shape)
for ii, index in enumerate(idx):
    shuffled_probe_features_batch1[ii, :, :] = probe_features_batch1_add0[index]

(124, 82, 1000)


In [23]:
batch_1_target_0 = np.concatenate((gallery_features_batch1_add0, shuffled_probe_features_batch1), axis=1)
target_0 = np.zeros([batch_1_target_0.shape[0]])

In [24]:
batch_1_target_0[1][82:]

array([[ 0.2820414 ,  0.31092837,  0.30656928, ...,  0.16087618,
         0.31453818,  0.39509431],
       [ 0.27821377,  0.28756857,  0.3085832 , ...,  0.1579583 ,
         0.29957786,  0.40155482],
       [ 0.29017514,  0.30860469,  0.32465994, ...,  0.15748253,
         0.28548372,  0.39662218],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [25]:
batch_1_target_1[116][82:]

array([[ 0.2820414 ,  0.31092837,  0.30656928, ...,  0.16087618,
         0.31453818,  0.39509431],
       [ 0.27821377,  0.28756857,  0.3085832 , ...,  0.1579583 ,
         0.29957786,  0.40155482],
       [ 0.29017514,  0.30860469,  0.32465994, ...,  0.15748253,
         0.28548372,  0.39662218],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
# Concatenate half_batch_1_target_0 and half_batch_1_target_1 together
new_batch_1 = np.concatenate((batch_1_target_1, batch_1_target_0), axis=0)
new_batch_1.shape

(248, 164, 1000)

In [27]:
with open('gait_data/casia_b_fc8_124/ordered_new_batch_1', 'wb') as f:
    np.save(f, new_batch_1)

In [28]:
target_1_and_0 = np.concatenate((target_1, target_0))

In [29]:
# Shuffle new_half_batch_1 to random order
idx2 = np.arange(len(new_batch_1))
np.random.shuffle(idx2)
idx2

array([113,  47, 104,   3,  59,  83,  78,  27,  56,   4, 191, 167, 207,
       214,  13, 166,  60,  57, 105,  49, 149, 209, 162,   0,  75,  64,
        84, 152, 129, 230,  16, 114, 238, 164, 192,  81, 201, 130, 234,
       116,  71, 115, 241,  15, 153, 198, 102, 123,  11,  14, 210, 227,
       106, 242, 160, 237, 195, 150,  92, 193, 165, 240, 184, 141,  39,
       177, 219, 127, 233,  30, 148, 188, 228,  61, 243, 244, 181, 118,
       204,  18,  48, 236, 111,  52,  24,  45, 187,  99, 155, 208, 205,
       235, 189, 203, 157,  85, 218, 156, 168,  35,  89, 161,  36,  51,
        79,  94, 246, 217,  25, 112, 206,  19, 125,  96, 199,  97, 142,
       202, 229, 136,  53,  76,  20, 223, 140,  95, 212,  72, 197,  86,
       139, 117,  66,  67, 171, 122, 120, 147, 100, 213,  28,  82, 145,
       134,   9,   7, 239, 215, 196, 176,  41, 135, 182,  77,  44,  69,
       138, 179,  21, 226, 183, 109, 159,   1, 175,  68, 225,  46,  17,
       119,  42, 126,  43, 144, 107,  65, 163, 178, 180, 185, 13

In [30]:
shuffled_new_batch_1 = np.ndarray(new_batch_1.shape, dtype=float)
shuffled_target_1_and_0 = np.ndarray(target_1_and_0.shape, dtype=float)
for ii, index in enumerate(idx2):
    shuffled_new_batch_1[ii, :, :] = new_batch_1[index]
    shuffled_target_1_and_0[ii] = target_1_and_0[index]

In [31]:
with open('gait_data/casia_b_fc8_124/shuffled_new_batch_1', 'wb') as f:
    np.save(f, shuffled_new_batch_1)

In [32]:
one_hot_shuffled_new_batch_1_target = np.ndarray([shuffled_target_1_and_0.shape[0], 2], dtype=float)
for ii, each in enumerate(shuffled_target_1_and_0):
    if int(each) == 1:
        one_hot_shuffled_new_batch_1_target[ii, :] = 0.9, 0.1
    elif int(each) == 0:
        one_hot_shuffled_new_batch_1_target[ii, :] = 0.1, 0.9

In [33]:
with open('gait_data/casia_b_fc8_124/one_hot_shuffled_new_batch_1_target', 'wb') as f:
    np.save(f, one_hot_shuffled_new_batch_1_target)

## Create seq_length

In [34]:
p_seq_length = np.array(p_seq_length)
shuffled_p_seq_length = np.zeros(p_seq_length.shape, dtype=int)
for ii, index in enumerate(idx):
    shuffled_p_seq_length[ii] = p_seq_length[index]

In [47]:
seq_column1 = np.reshape(np.concatenate([g_seq_length, p_seq_length], axis=0), [len(g_seq_length)*2, 1])
seq_column2 = np.reshape(np.concatenate([g_seq_length, shuffled_p_seq_length], axis=0), [len(g_seq_length)*2, 1])
seq_length = np.concatenate([seq_column1, seq_column2], axis=1)

shuffled_seq_length = np.zeros(seq_length.shape, dtype=int)
for ii, index in enumerate(idx2):
    shuffled_seq_length[ii] = seq_length[index]

In [49]:
with open('gait_data/casia_b_fc8_124/shuffled_seq_length', 'wb') as f:
    np.save(f, shuffled_seq_length)

# Checkpoints 1

In [10]:
seq_length = np.load(open(r'../gait_data/casia_b_fc8_124/degree_72_normal_walk/shuffled_seq_length', mode='rb'))
shuffled_new_batch_1 = np.load(open(r'../gait_data/casia_b_fc8_124/degree_72_normal_walk/shuffled_new_batch_1', mode='rb'))
one_hot_shuffled_new_batch_1_target = np.load(open(r'../gait_data/casia_b_fc8_124/degree_72_normal_walk/one_hot_shuffled_new_batch_1_target', mode='rb'))
print(shuffled_new_batch_1.shape)

(248, 164, 1000)


# Get Train and Validation data

In [11]:
print("data length: {}".format(len(shuffled_new_batch_1)))
print("every length has : {}".format(shuffled_new_batch_1.shape))

data length: 248
every length has : (248, 164, 1000)


In [12]:
split_frac = 0.8
split_num = int(split_frac*len(shuffled_new_batch_1))

train_x_raw, val_x_raw = shuffled_new_batch_1[:split_num], shuffled_new_batch_1[split_num:]
train_y_raw, val_y_raw = one_hot_shuffled_new_batch_1_target[:split_num], one_hot_shuffled_new_batch_1_target[split_num:]
train_seq_length, val_seq_length = seq_length[:split_num], seq_length[split_num:]

# print size
print("Train_x: {}".format(np.array(train_x_raw).shape))
print("Validation_x: {}".format(np.array(val_x_raw).shape))
print("Train_y: {}".format(np.array(train_y_raw).shape))
print("Validation_y: {}".format(np.array(val_y_raw).shape))
print("train_seq_length: {}".format(np.array(train_seq_length).shape))
print("val_seq_length: {}".format(np.array(val_seq_length).shape))

Train_x: (198, 164, 1000)
Validation_x: (50, 164, 1000)
Train_y: (198, 2)
Validation_y: (50, 2)
train_seq_length: (198, 2)
val_seq_length: (50, 2)


# According_len_get_outputs

In [13]:
def According_len_get_outputs(length_batch_1, length_batch_2, outputs_1, outputs_2):
    """
    length_batch_1 and length_batch_2 should minus one
    """
    new_outputs_1 = np.ndarray([outputs_1.shape[0], outputs_1.shape[2]], dtype=float)
    new_outputs_2 = np.ndarray([outputs_2.shape[0], outputs_2.shape[2]], dtype=float)
    
    for ii, (L1, L2, O1, O2) in enumerate(zip(length_batch_1, length_batch_2, outputs_1, outputs_2)):
        new_outputs_1[ii, :] = O1[L1, :]
        new_outputs_2[ii, :] = O2[L2, :]
        
    return new_outputs_1, new_outputs_2

# Build the graph

In [26]:
def build_rnn(batch_size=20, lstm_size=1000, num_layers=1, learning_rate=0.001, \
              grad_clip=1, alpha=0.2):
    
    # reset graph
    tf.reset_default_graph()
    
    with tf.name_scope('inputs_1'):
        inputs_1 = tf.placeholder(tf.float32, [None, None, 1000], name="inputs_1")
    with tf.name_scope('inputs_2'):
        inputs_2 = tf.placeholder(tf.float32, [None, None, 1000], name="inputs_2")
    with tf.name_scope('targets'):
        target_ = tf.placeholder(tf.float32, [None, 2], name="targets")
        
    seq_1 = tf.placeholder(tf.float32, (None,), name="seq_1")
    seq_2 = tf.placeholder(tf.float32, (None,), name="seq_2")
    
#     new_outputs_1 = tf.placeholder(tf.float32, [None, lstm_size], name="new_outputs_1")
#     new_outputs_2 = tf.placeholder(tf.float32, [None, lstm_size], name="new_outputs_2")
    
#     new_final_state_1 = tf.placeholder(tf.float32, [None, lstm_size], name="new_final_state_1")
#     new_final_state_2 = tf.placeholder(tf.float32, [None, lstm_size], name="new_final_state_2")
    final_state_concat = tf.placeholder(tf.float32, [None, int(np.sqrt(lstm_size)), int(np.sqrt(lstm_size)), 1], \
                                       name='final_state_concat')
    
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    is_training = tf.placeholder(tf.bool, name='is_training')
    
    with tf.variable_scope("LSTM"):
#         with tf.name_scope("RNN_cells_1"):
            # Your basic LSTM cell
        #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
        #         lstm = tf.nn.relu(lstm)
        lstm_1 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
#         batch_norm_1 = tf.layers.batch_normalization(lstm_1, reuse=True)
            # Add dropout to the cell
        drop_1 = tf.contrib.rnn.DropoutWrapper(lstm_1, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_1 = tf.contrib.rnn.MultiRNNCell([drop_1] * num_layers)

#         with tf.name_scope("RNN_init_state_1"):
            # Getting an initial state of all zeros
        initial_state_1 = cell_1.zero_state(batch_size, tf.float32)
#         with tf.name_scope("RNN_forward_1"):
        outputs_1, final_state_1 = tf.nn.dynamic_rnn(cell_1, inputs_1, initial_state=initial_state_1, 
                                                     sequence_length=seq_1)

    with tf.variable_scope("LSTM", reuse=True):
#         with tf.name_scope("RNN_cells_2"):
            # Your basic LSTM cell
    #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
    #         lstm = tf.nn.relu(lstm)
        lstm_2 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
#         batch_norm_2 = tf.layers.batch_normalization(lstm_2, reuse=True)
            # Add dropout to the cell
        drop_2 = tf.contrib.rnn.DropoutWrapper(lstm_2, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_2 = tf.contrib.rnn.MultiRNNCell([drop_2] * num_layers)

#         with tf.name_scope("RNN_init_state_2"):
            # Getting an initial state of all zeros
        initial_state_2 = cell_2.zero_state(batch_size, tf.float32)

#         with tf.name_scope("RNN_forward_2"):
        # shape [1, 2, batch_size, lstm_size]
        outputs_2, final_state_2 = tf.nn.dynamic_rnn(cell_2, inputs_2, initial_state=initial_state_2, 
                                                     sequence_length=seq_2)

    with tf.name_scope('predictions'):
        # only choose final one
#         final_output_concat = tf.concat([outputs_1[:, -1], outputs_2[:, -1]], 1)

        # through the According_len_get_outputs
    
        # subtraction
#         final_output_concat = abs(new_final_state_1 - new_final_state_2)
#         final_output_concat = abs(new_outputs_1 - new_outputs_2)
        
#         # concatenation
#         final_output_concat = tf.concat([new_outputs_1, new_outputs_2], 1)

        # Conv pool_1
        conv_1 = tf.layers.conv2d(final_state_concat, 16, 2, strides=2, padding='same')
#         conv_1 = tf.maximum(alpha * conv_1, conv_1)
        conv_1 = tf.nn.relu(conv_1)
        pool_1 = tf.nn.max_pool(conv_1, [1,2,2,1], [1,2,2,1], padding='SAME')
        
        # Conv pool_2
        conv_2 = tf.layers.conv2d(pool_1, 32, 2, strides=2, padding='same')
        conv_2 = tf.layers.batch_normalization(conv_2, training=is_training)
#         conv_2 = tf.maximum(alpha * conv_2, conv_2)
        conv_2 = tf.nn.relu(conv_2)
        pool_2 = tf.nn.max_pool(conv_2, [1,2,2,1], [1,2,2,1], padding='SAME')
        
        # Conv pool_3
        conv_3 = tf.layers.conv2d(pool_2, 64, 2, strides=2, padding='same')
        conv_3 = tf.layers.batch_normalization(conv_3, training=is_training)
#         conv_3 = tf.maximum(alpha * conv_3, conv_3)
        conv_3 = tf.nn.relu(conv_3)
        pool_3 = tf.nn.max_pool(conv_3, [1,2,2,1], [1,2,2,1], padding='SAME')
        
        # Flatten
        pool_3 = tf.reshape(pool_3, [-1, pool_3.get_shape().as_list()[1]*pool_3.get_shape().as_list()[2]*\
                                     pool_3.get_shape().as_list()[3]])
        
        # Fully_connected
        f_1 = tf.contrib.layers.fully_connected(pool_3, 100)
        logits = tf.contrib.layers.fully_connected(f_1, 2)
        predictions = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', predictions)
        
#     with tf.name_scope('accuracy'):
#         correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(target_, 1))
#         accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

    with tf.name_scope('cost'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target_, logits=logits))
        tf.summary.scalar('cost', cost)
        
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip, name='clip')
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
#         optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        
    with tf.name_scope('saver'):
        saver = tf.train.Saver()
        
    merged = tf.summary.merge_all()
    
    export_nodes = ['inputs_1', 'inputs_2', 'target_', 'initial_state_1', 'initial_state_2', 'outputs_1', 'outputs_2',
                    'final_state_1', 'final_state_2', 'keep_prob', 'cost', 'logits', 'predictions', 'optimizer', 
                    'merged', 'saver', 'learning_rate', 'cell_1', 'cell_2', 'seq_1', 'seq_2', 'final_state_concat',
                   'is_training']
    
    Graph = collections.namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

# Get Batches

In [27]:
def get_batches(data, target, seq_length, batch_size):
    n_batches = len(data)//batch_size
    data = data[:batch_size * n_batches]
    target = target[:batch_size * n_batches]
    for ii in range(0, batch_size*n_batches, batch_size):
        data_batch = data[ii:ii + batch_size]
        target_batch = target[ii:ii + batch_size]
        seq_length_batch_1 = seq_length[ii:ii + batch_size, 0]
        seq_length_batch_2 = seq_length[ii:ii + batch_size, 1]
        
        yield data_batch, target_batch, seq_length_batch_1, seq_length_batch_2

# Train

In [28]:
train_x = train_x_raw
val_x = val_x_raw
train_y = train_y_raw
val_y = val_y_raw

In [29]:
train_x.shape

(198, 164, 1000)

# Accuracy

In [30]:
def calculate_acc(pred, target):
    correct_pred = np.equal(np.argmax(pred, axis=1), np.argmax(target, axis=1))
    acc = np.mean(correct_pred.astype(float)) * 100
    
    return acc

In [58]:
def train(model, epochs, file_writer, save_string):

    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(tf.global_variables_initializer())
        
        iteration = 1
        mean_val_Acc = 0
        mean_val_loss = 0
        count_Acc_not_increase_epochs = 0
        count_loss_not_decrease_epochs = 0
        Last_val_Acc = 0
        Last_val_loss = 0
        
        
        
        for e in range(epochs):
            for train_batch, target_batch, seq_length_batch_1, seq_length_batch_2 \
                             in get_batches(train_x, train_y, train_seq_length, batch_size):
                    
                start = time.time()
                
#                 print(train_batch[:, :15, :].shape)
#                 print(target_batch)
    #             print('target: {}'.format(target_batch.shape))
                state_1 = sess.run(model.cell_1.zero_state(batch_size, tf.float32))
                state_2 = sess.run(model.cell_2.zero_state(batch_size, tf.float32))

                feed_1 = {
                        model.inputs_1: train_batch[:, :82, :],
                        model.inputs_2: train_batch[:, 82:, :],  
                        model.seq_1: seq_length_batch_1,
                        model.seq_2: seq_length_batch_2,
                        model.keep_prob: 0.5,
                        model.initial_state_1: state_1,
                        model.initial_state_2: state_2,
                        model.is_training: True
                        }
            
#                 print(model.outputs_1.shape, model.outputs_1[:, -1], model.final_output_concat, 
#                       model.logits, model.predictions, model.cost)
                f_1, f_2 = sess.run([model.final_state_1, model.final_state_2], feed_dict=feed_1)
                
#                 state_1 = f_1
#                 state_2 = f_2

                show_f_state_1 = np.reshape(f_1[0][1], [batch_size, \
                                                        int(np.sqrt(lstm_size)), int(np.sqrt(lstm_size)), 1])
                show_f_state_2 = np.reshape(f_2[0][1], [batch_size, \
                                                        int(np.sqrt(lstm_size)), int(np.sqrt(lstm_size)), 1])
                show_final_state = abs(show_f_state_1 - show_f_state_2)
#                 print('show_output:', target_batch[1])
#                 plt.imshow(np.reshape(show_final_state[1], [int(np.sqrt(lstm_size)), int(np.sqrt(lstm_size))]))
#                 plt.show()
#                 real_outputs_1, real_outputs_2 = According_len_get_outputs(
#                                                seq_length_batch_1 - 1, seq_length_batch_2 - 1, o_1, o_2)
            
#                 show_output_1 = np.reshape(real_outputs_1[0], [50, 60])
#                 show_output_2 = np.reshape(real_outputs_2[0], [50, 60])
#                 print('show_output:', target_batch[0])
#                 plt.imshow(abs(show_output_1 - show_output_2))
#                 plt.show()
    
#                 print(o_1[:, 28], o_2[:, 28])
#                 print(real_outputs_1, real_outputs_2)
                feed_2 = {
                        model.target_: target_batch,
                        model.keep_prob: 0.5,
#                         model.initial_state_1: state_1,
#                         model.initial_state_2: state_2,
                        model.learning_rate: lr,
                        model.final_state_concat: show_final_state,
                        model.is_training: True
#                         model.new_outputs_1: real_outputs_1,
#                         model.new_outputs_2: real_outputs_2
                        }
                
                loss, _, logi, pred, summary = sess.run(
                                [model.cost, model.optimizer, model.logits, model.predictions, model.merged]
                                 , feed_dict=feed_2)
            
                Acc = calculate_acc(pred, target_batch)
                
#                 print(pred)
#                 print(logi)
#                 print('pred: {}'.format(type(pred)))
    #             print('pred: {}'.format(np.count_nonzero(pred)))
    #             print('pred: {}'.format(np.sum(pred>0.1)))
    #             print('pred: {}'.format(pred.size))
    #             print('output: {}'.format(out[1, 39, :]))
#                 print(pred.shape)
        
                
                if iteration%5==0:
                    end = time.time()
#                     acc = calculate_accuracy(sess, pred, target_batch, sq_length)
                    print(
                          "Epoch: {}/{},".format(e+1, epochs),' ',
                          "Iteration: {},".format(iteration),' ',
                          "Train loss: {:.3f},".format(loss),' ',
                          "{:.1f}s /batch.".format((end-start)/5),' '
                          "ACCuracy: %{}".format(Acc)
                         )
                    
                    file_writer.add_summary(summary, iteration)
            
                ##############################################################
                ######################## VALIDATION ##########################
                ##############################################################
                
                if iteration%15==0:
                    training = True
                    validation_loss = []
                    validation_Acc = []
                    
                    if batch_size >= len(val_x):
                        val_batch_size = 10
                    else: 
                        val_batch_size = batch_size
                                            
                    for val_batch, val_target_batch, val_seq_length_batch_1, val_seq_length_batch_2 \
                             in get_batches(val_x, val_y, val_seq_length, val_batch_size):
                            
                        val_state_1 = sess.run(model.cell_1.zero_state(val_batch_size, tf.float32))
                        val_state_2 = sess.run(model.cell_2.zero_state(val_batch_size, tf.float32))
#                         print(val_state_1[0][0].shape)

                        feed_3 = {
                                model.inputs_1: val_batch[:, :82, :],
                                model.inputs_2: val_batch[:, 82:, :],  
                                model.seq_1: val_seq_length_batch_1,
                                model.seq_2: val_seq_length_batch_2,
                                model.keep_prob: 1,
                                model.initial_state_1: val_state_1,
                                model.initial_state_2: val_state_2,
                                model.is_training: True
                                }
                        
                        val_f_1, val_f_2 = sess.run([model.final_state_1, model.final_state_2], feed_dict=feed_3)
                        
                        val_show_f_state_1 = np.reshape(val_f_1[0][1], [val_batch_size, \
                                                        int(np.sqrt(lstm_size)), int(np.sqrt(lstm_size)), 1])
                        val_show_f_state_2 = np.reshape(val_f_2[0][1], [val_batch_size, \
                                                        int(np.sqrt(lstm_size)), int(np.sqrt(lstm_size)), 1])
                        val_show_final_state = abs(val_show_f_state_1 - val_show_f_state_2)
                        
                        feed_4 = {
                                model.target_: val_target_batch,
                                model.keep_prob: 1,
#                         model.initial_state_1: state_1,
#                         model.initial_state_2: state_2,
                                model.learning_rate: lr,
                                model.final_state_concat: val_show_final_state,
                                model.is_training: True
#                         model.new_outputs_1: real_outputs_1,
#                         model.new_outputs_2: real_outputs_2
                                 }
                        
                        val_loss, val_pred = sess.run([model.cost, model.predictions], feed_dict=feed_4)
                        
                        val_acc = calculate_acc(val_pred, val_target_batch)

                        validation_loss.append(val_loss)
                        
                        validation_Acc.append(val_acc)
                        
                    Last_val_Acc = mean_val_Acc
                    Last_val_loss = mean_val_loss
                        
                    mean_val_loss = sum(validation_loss)/len(validation_loss)
                    mean_val_Acc = sum(validation_Acc)/len(validation_Acc)
                    print()
                    print("Validation loss: {:.3f},".format(mean_val_loss), ' ',
                          "Validation Accuracy: %{:.3f}".format(mean_val_Acc))
                    print()
                    
                iteration += 1
                
#             #  Early stopping  
#             if mean_val_Acc - Last_val_Acc <= -0.5:
#                 count_Acc_not_increase_epochs += 1
#             if Last_val_loss - mean_val_loss <= -0.03:
#                 count_loss_not_decrease_epochs += 1
                
#             if mean_val_Acc - Last_val_Acc <= -2:
#                 break
#             if count_Acc_not_increase_epochs >= 50:
#                 break
#             if count_loss_not_decrease_epochs >= 50:
#                 break
                
        model.saver.save(sess, r"{}".format(save_string))

In [59]:
!mkdir checkpoints_identification

mkdir: checkpoints_identification: File exists


In [60]:
training = True
degree = 72
epochs = 1500
batch_size = 30
for lstm_size in [625]: # 625 91%
    for num_layers in [1]:
        for lr in [0.001]:
                log_string = 'logs/4/dg={},lr={},nl={},ls={},bs={}'.format(degree, lr, num_layers, lstm_size, batch_size)
                save_string = 'checkpoints\dg={}_lr={}_nl={}_ls={}_bs={}.ckpt'.format(degree, lr, num_layers, lstm_size, batch_size)
                
                writer = tf.summary.FileWriter(log_string)
                                
                model = build_rnn(batch_size=batch_size, lstm_size=lstm_size, \
                                  num_layers=num_layers, learning_rate=lr, grad_clip=1)

                train(model, epochs, writer, save_string)
                
                print(' ')
                print(' ')
                print("leraning_rate={},num_layers={},lstm_size={},batch_size={} finished, saved".format(lr, num_layers, lstm_size, batch_size))
                print(' ')
                print(' ')

Epoch: 1/1500,   Iteration: 5,   Train loss: 0.709,   0.3s /batch.  ACCuracy: %40.0
Epoch: 2/1500,   Iteration: 10,   Train loss: 0.650,   0.4s /batch.  ACCuracy: %56.666666666666664
Epoch: 3/1500,   Iteration: 15,   Train loss: 0.574,   0.4s /batch.  ACCuracy: %80.0

Validation loss: 0.617,   Validation Accuracy: %73.333

Epoch: 4/1500,   Iteration: 20,   Train loss: 0.542,   0.3s /batch.  ACCuracy: %80.0
Epoch: 5/1500,   Iteration: 25,   Train loss: 0.547,   0.3s /batch.  ACCuracy: %90.0
Epoch: 5/1500,   Iteration: 30,   Train loss: 0.497,   0.3s /batch.  ACCuracy: %83.33333333333334

Validation loss: 0.591,   Validation Accuracy: %73.333

Epoch: 6/1500,   Iteration: 35,   Train loss: 0.515,   0.3s /batch.  ACCuracy: %80.0
Epoch: 7/1500,   Iteration: 40,   Train loss: 0.484,   0.3s /batch.  ACCuracy: %86.66666666666667
Epoch: 8/1500,   Iteration: 45,   Train loss: 0.426,   0.3s /batch.  ACCuracy: %93.33333333333333

Validation loss: 0.495,   Validation Accuracy: %80.000

Epoch: 9/150

KeyboardInterrupt: 